In [ ]:
###find number of mowings in each shapefile###
import fiona
import matplotlib.pyplot as plt
import numpy as np
area = 6
sum_of_areas = 0
sum_of_events = 0
for i in range(1,area+1):
    shapefile_path = f'/mowing_annotation_{i}.shp'
    # List for count
    count_values = []
    with fiona.open(shapefile_path, "r") as shapefile:
        count_values = [feature["properties"]["count"] for feature in shapefile]
    unique_numbers, counts = np.unique(count_values, return_counts=True)
    print(f'Sum of mowned areas for area {i}',sum(counts))
    print(f'Sum of mowing events for area {i}',sum(count_values))
    sum_of_areas += sum(counts)
    sum_of_events += sum(count_values)
    plt.bar(unique_numbers, counts, color='blue', alpha=0.7)
    plt.xlabel('Number of Events')
    plt.ylabel('Counts')
    plt.title(f'Mowing Event per Class in area {i}')
    plt.yticks(np.arange(min(counts), max(counts)+1, 4))
    plt.show()
    
print('Sum of mowned areas',sum_of_areas)
print('Sum of mowing events',sum_of_events)

In [ ]:
shapefile_path = f'/mowing_annotation_{17}.shp'
    # List for count
count_values = []
with fiona.open(shapefile_path, "r") as shapefile:
    count_values = [feature["properties"]["count"] for feature in shapefile]
unique_numbers, counts = np.unique(count_values, return_counts=True)
print(f'Sum of mowned areas for area {i}',sum(counts))
print(f'Sum of mowing events for area {i}',sum(count_values))
sum_of_areas += sum(counts)
sum_of_events += sum(count_values)
plt.bar(unique_numbers, counts, color='blue', alpha=0.7)
plt.xlabel('Number of Events')
plt.ylabel('Counts')
plt.title(f'Mowing Event per Class in area {i}')
plt.yticks(np.arange(min(counts), max(counts)+1, 4))
plt.show()
    
print('Sum of mowned areas',sum_of_areas)
print('Sum of mowing events',sum_of_events)

In [ ]:
###iterate through a dict and values and save each polygon to a folder associatedd with its class###
import fiona
import rasterio
import rasterio.mask
import numpy as np
import os
import json

input_folder = '/data/'
export_folder_path = '/parcel_level_data/'

INDEX_DESCRTPTION = 'SAR'
area_dict = {'1':[0,1,2,3,4],'2':[0,1,2,3,4],
            '3':[0,1,2,3,4],'4':[0,1,2,3,4],
            '5':[0,1,2,3,4],'6':[0,1,2,3,4]}


for i,key in enumerate(area_dict):
    print('area is',key)
    values = area_dict[key]
    print('values are',values)
    for j,value in enumerate(values):
        area = key
        target_attribute_value = value
        shapefile_path = f'/mowing_annotation_{area}.shp'

        with fiona.open(shapefile_path, "r") as shapefile:

            selected_shapes = [feature["geometry"] for feature in shapefile if feature["properties"]["count"] == target_attribute_value]

        for k,shape in enumerate(selected_shapes):

            with rasterio.open(input_folder+f'kampos_collective_{area}/kampos_{area}_image_export{INDEX_DESCRTPTION}.tif') as src:
                # Mask the raster based on the selected shapefile geometries
                print('entered rasterio')
                out_image, out_transform = rasterio.mask.mask(src, [shape], crop=True, nodata=np.nan)
                out_meta = src.meta  # Get metadata from the source raster
                
            # Update metadata for the output raster
            out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform})
            #break
            print(export_folder_path+str(target_attribute_value)+'/'+str(i)+str(j)+str(k)+'.tif')
            output_raster_path = export_folder_path+str(target_attribute_value)+'/'+str(i)+str(j)+str(k)+'.tif'
            with rasterio.open(output_raster_path, "w", **out_meta) as dest:
                dest.write(out_image)
            
            



In [ ]:
###iterate through a dict and values and save each polygon to a folder associatedd with its class for pretrain data###
import fiona
import rasterio
import rasterio.mask
import numpy as np
import os
import json
from tqdm import tqdm

###change folder, where the ts is saved###
input_folder = '/area_4_preproc/'
###change folder, where the data must be saved saved###
export_folder_path = '/pretrain_area_4/'
###change the index according to the file name###
INDEX_DESCRTPTION = 'ts_area_4'

area_dict = {'17':[0]}
#area_dict = {'17':[0,1,2,3,4]}

for i,key in enumerate(area_dict):
    print('area is',key)
    values = area_dict[key]
    print('values are',values)
    for j,value in enumerate(values):
        area = key
        target_attribute_value = value
        ###enter the right shapefile name###
        shapefile_path = f'/2021_ger_sieved_{area}_clipped_30_shape.shp'

        with fiona.open(shapefile_path, "r") as shapefile:
            selected_shapes = [feature["geometry"] for feature in shapefile if feature["properties"]["count"] == target_attribute_value]
            #print(selected_shapes)
       
        for k,shape in enumerate(tqdm(selected_shapes)):
            #print(shape.keys(),'area =',area,'target_attribute_value =',target_attribute_value)
            #print(type(shape))
            #print(shape) 
            #shape = json.dumps(shape, ensure_ascii=False)
            #print(list[shape]) 
            
            #print(export_folder_path+str(target_attribute_value)+'/'+str(i)+str(j)+str(k))
            ###enter the right timeseries tif file name###
            with rasterio.open(input_folder+f'thunen_{area}_{INDEX_DESCRTPTION}.tif') as src:
                # Mask the raster based on the selected shapefile geometries
                #print('entered rasterio')
                out_image, out_transform = rasterio.mask.mask(src, [shape], crop=True, nodata=np.nan)
                out_meta = src.meta  # Get metadata from the source raster
                
            
            out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform})
            #break
            #print(export_folder_path+str(target_attribute_value)+'/'+str(i)+str(j)+str(k)+'.tif')
            output_raster_path = export_folder_path+str(target_attribute_value)+'/'+str(i)+str(j)+str(k)+'.tif'
            with rasterio.open(output_raster_path, "w", **out_meta) as dest:
                dest.write(out_image)
            
            



In [ ]:
###iterate through a dict and values and save each polygon to a folder associatedd with its class for HLS data###
import fiona
import rasterio
import rasterio.mask
import numpy as np
import os
import json
from tqdm import tqdm


# input folder for the timeseries data to crop
input_folder = '/timeseries_areas/'


## export folder for the dataset##
##each different class--> different folder##
export_folder_path = '/hls_data_mix/'
INDEX_DESCRTPTION = '_full_hls'
#'WNDVI'
area_dict = {'17':[0,1,2,3,4]}


for i,key in enumerate(area_dict):
    print('area is',key)
    values = area_dict[key]
    print('values are',values)
    for j,value in enumerate(values):
        area = key
        target_attribute_value = value
        shapefile_path = f'/mowing_annotation_hls_{area}.shp'

        with fiona.open(shapefile_path, "r") as shapefile:
            selected_shapes = [feature["geometry"] for feature in shapefile if feature["properties"]["count"] == target_attribute_value]
            
       
        for k,shape in enumerate(selected_shapes):

            with rasterio.open(input_folder+f'kampos_{area}_image_export{INDEX_DESCRTPTION}.tif') as src:

                out_image, out_transform = rasterio.mask.mask(src, [shape], crop=True, nodata=np.nan)
                out_meta = src.meta  
                
            
            out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform})

            output_raster_path = export_folder_path+str(target_attribute_value)+'/'+str(i)+str(j)+str(k)+'.tif'
            with rasterio.open(output_raster_path, "w", **out_meta) as dest:
                dest.write(out_image)
            
            



In [ ]:
from tqdm import tqdm

In [ ]:
###json of labels as in previous###
###create a list of dicts with available data###
import os
import json

def create_class_filename_json(data_path, output_file='../gt_jsons/myfile_area_4.json'):
    class_filename_list = []
    for class_folder in os.listdir(data_path):
        class_path = os.path.join(data_path, class_folder)

        if os.path.isdir(class_path):
            for filename in os.listdir(class_path):
                if filename.endswith('.tif'):
                    class_filename_list.append({os.path.splitext(filename)[0]: class_folder})
    #print(class_filename_list)
    with open(output_file, "w") as out_file:
        json.dump(class_filename_list, out_file)
    

data_path = '/pretrain_area_4/'
result_dict = create_class_filename_json(data_path)


In [ ]:
###here we do the split for each area based on the area_X###
import json

for i in range(1,7):
    
    file_path = 'channels.json'
    with open(file_path, 'r') as file:
        original_json = json.load(file)
    
    
    
    print('the len of original json is',len(original_json))
    
    
    desired_area = f'area_{i}'
    

    filtered_data = [{key: value} for item in original_json for key, value in item.items() if key.startswith(desired_area)]
    print('the len of the filtered data is', len(filtered_data))
    
    output_file_path = f'area_split_{desired_area}_val.json'
    print('saving filtered in',output_file_path)
    
    #print(filtered_data)
    
    with open(output_file_path, 'w') as output_file:
        json.dump(filtered_data, output_file, indent=2)
    
    print(f"Filtered data for {desired_area} saved to {output_file_path}")
    
    print('entering train split section')
    
    filtered_train = []
    for entry in original_json:
        entry_keys = entry.keys()
        if any(desired_area not in key for key in entry_keys):
            print(f"NOT Found {desired_area} in keys for entry:", entry)
            filtered_train.append(entry)
        

    
    output_file_path_train = f'area_split_{desired_area}_train.json'
    
    print('saving train split to path', output_file_path_train)

    with open(output_file_path_train, 'w') as output_file:
        json.dump(filtered_train, output_file, indent=2)


In [ ]:
###create the train json that does not contain the validation split###
file_path = 'myfile_area_split_all.json'
desired_area = 'area_5'


# Read JSON data from the file
with open(file_path, 'r') as file:
    original_json = json.load(file)

filtered = []
for entry in original_json:
    entry_keys = entry.keys()
    if any(desired_area not in key for key in entry_keys):
        print(f"Found {desired_area} in keys for entry:", entry)
        filtered.append(entry)
    

print(filtered)


In [ ]:
###split the files###
import json
from sklearn.utils import shuffle


with open('../gt_jsons/myfile_thunen_area_4.json', 'r') as file:
    json_data = file.read()
    
data_dict = json.loads(json_data)
result_list = shuffle(data_dict)
train_ratio = 0.8  
val_ratio = 0.1
test_ratio = 0.1  

total_size = len(result_list)
train_size = int(total_size * train_ratio)
val_size = int(total_size * val_ratio)

train_data = result_list[:train_size]
val_data = result_list[train_size:train_size + val_size]
test_data = result_list[train_size + val_size:]


train_data = train_data
val_data = val_data
test_data = test_data

print('length of train is',len(train_data))
print('length of validation is',len(val_data))
print('length of test is',len(test_data))



with open('../gt_jsons/train_thunen_area_4.json', 'w') as file:
    json.dump(train_data, file, ensure_ascii=False)

with open('../gt_jsons/val_thunen_area_4.json', 'w') as file:
    json.dump(val_data, file, ensure_ascii=False)

with open('../gt_jsons/test_thunen_area_4.json', 'w') as file:
    json.dump(test_data, file, ensure_ascii=False)



#with open('kfold.json', 'w') as f:
#    json.dump(result_list, f)
        
    
